In [59]:
%%writefile language_app.py

import streamlit as st  # Streamlit is the framework used to create the web app
import base64  # Used to handle encoding of the background image
from transformers import MarianMTModel, MarianTokenizer  # Used for translation (commented out for future potential use)
from deep_translator import GoogleTranslator  # A library used for translation via the Google Translate API
import re  # Regular expressions library for text preprocessing
import string  # Used to handle string operations
import nltk  # Natural Language Toolkit, used for text preprocessing
from nltk.tokenize import word_tokenize  # Function to tokenize text into words

# Set the NLTK data path to your local directory (avoiding the need to download data again)
nltk.data.path.append('/Users/manisunder/nltk_data/nltk_data')

# Text Preprocessing Functions
# These functions help clean and prepare the input text before translation

def normalize_text(text):
    text = text.lower()  # Converts all characters to lowercase for consistency
    return text

def remove_special_characters(text):
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Removes punctuation and special characters
    return text

def tokenize_text(text):
    tokens = word_tokenize(text)  # Breaks the text into individual words (tokens)
    return " ".join(tokens)  # Joins the tokens back into a single string

def preprocess_text(text):
    text = normalize_text(text)  # Step 1: Normalize the text
    text = remove_special_characters(text)  # Step 2: Remove special characters
    text = tokenize_text(text)  # Step 3: Tokenize the text
    return text

# Functions to handle the background image
def get_base64_of_bin_file(bin_file):
    with open(bin_file, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()  # Encode the image file as base64 for web usage

def set_png_as_page_bg(png_file):
    bin_str = get_base64_of_bin_file(png_file)  # Convert the image file to base64
    # Insert custom CSS to apply the background image and style the page
    page_bg_img = f'''
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap');

    body {{
    background-image: url("data:image/png;base64,{bin_str}");
    background-size: cover;
    background-position: center;
    background-repeat: no-repeat;
    background-attachment: fixed;
    opacity: 0.9;
    font-family: 'Playfair Display', serif;
    }}

    .stApp {{
    background: rgba(255, 255, 255, 0.99); /* Add a light background to content */
    z-index: 999; /* Ensure content is on top */
    font-family: 'Playfair Display', serif;
    }}

    [data-testid="stHeader"], .stToolbar {{
    background: rgba(0, 0, 0, 0) !important; /* Remove the white block at the top */
    }}

    input {{
    background-color: rgba(255, 255, 255, 0.9) !important; /* Background color for chat bar */
    border: 2px solid #ff6347; /* Add a border around the chat bar */
    color: black !important; /* Text color inside the chat bar */
    font-family: 'Playfair Display', serif; 
    }}

    </style>
    '''
    
    st.markdown(page_bg_img, unsafe_allow_html=True)  # Apply the custom CSS to the page
    return

# Set the background image for the app using the provided image file path
set_png_as_page_bg('/Users/manisunder/Desktop/DEGREES & CERTS/MTECH/AI-ML/SEM-3/NLP Apps/Assignment-2/Image.png')

# Add custom CSS for fonts and borders
st.markdown(
    """
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Great+Vibes&family=Courgette&display=swap');
    
    /* Title font */
    .title-font {
        font-family: 'Great Vibes', cursive;
        font-size: 48px;
        font-weight: bold;
    }
    
    /* Subtitle font for instructions */
    .subtitle-font {
        font-family: 'Tangerine', cursive;
        padding-top: 0rem;
        font-size: 24px;
        font-weight: bold;
        padding-bottom: 2rem;
    }
    
    /* General text styling */
    .courgette-font {
        font-family: 'Tangerine', cursive;
        padding-bottom: -5rem;
        padding-top: 1rem;
    }
    
    /* Remove padding at the top of the main content container */
    .block-container {
        padding-top: 0rem;
    }

    /* Styling for text inputs, text areas, and file uploaders */
    .stTextInput, .stTextArea, .stFileUploader{
        font-family: 'Tangerine', cursive;
    }
    
    /* Styling for the select box (dropdown menu) */
    select {
        border: 2px solid #007bff !important; /* Blue border */
        border-radius: 5px !important;
        font-family: 'Courgette', cursive !important;
    }
    
    </style>
    """,
    unsafe_allow_html=True
)

# Title of the app with custom font
st.markdown('<h1 class="title-font">Pan-Indian Language App</h1>', unsafe_allow_html=True)

# Instructions for the user with custom styling
st.markdown('<div class="subtitle-font">Input your text or upload a file</div>', unsafe_allow_html=True)

# Text input area with custom height and styling
text_input = st.text_area("Enter Text for translation", height=70)

# File uploader with custom styling
uploaded_file = st.file_uploader("Or Upload Text File", type=["txt"])

# Dropdown menu for selecting the target Indic language
language_options = {
    'Hindi': 'hi',
    'Tamil': 'ta',
    'Bengali': 'bn',
    'Telugu': 'te',
    'Marathi': 'mr'
}
target_language_name = st.selectbox("Select target language", list(language_options.keys()))
target_language_code = language_options[target_language_name]  # Retrieve the language code for translation

# Button to trigger the translation process
if st.button("Translate"):
    if text_input or uploaded_file:  # Check if either text input or file upload is provided
        if uploaded_file:
            text_input = uploaded_file.read().decode("utf-8")  # Read and decode the uploaded file
        
        # Preprocess the text before translation
        preprocessed_text = preprocess_text(text_input)
        
        try:
            # Perform the translation using the Google Translate API
            translated_text = GoogleTranslator(source='en', target=target_language_code).translate(preprocessed_text)
            
            # Display the translated text with custom styling
            st.markdown('<div class="courgette-font"><h2>Translated Text</h2></div>', unsafe_allow_html=True)
            st.markdown(f'<div class="courgette-font">{translated_text}</div>', unsafe_allow_html=True)
            
            # Create a downloadable link for the translated text
            st.download_button(
                label="Download Translated Text",
                data=translated_text,
                file_name="translated_text.txt",
                mime="text/plain"
            )
            
        except Exception as e:
            st.error(f"Error during translation: {e}")  # Display an error message if the translation fails
    else:
        st.warning("Please enter text or upload a file for translation.")  # Warn the user if no input is provided


Overwriting language_app.py
